In [ ]:
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
url="https://search.danawa.com/dsearch.php?query={}&checkedInfo=N&page={}&limit=40&sort=saveDESC&list=list&maxPrice=200000&tab=main".format("블루투스 헤드폰",1)
browser.get(url=url)

In [ ]:
def get_page(query,page):
  return "https://search.danawa.com/dsearch.php?query={}&checkedInfo=N&page={}&limit=40&sort=saveDESC&list=list&tab=main".format(query,page)

In [ ]:
html = browser.page_source
soup = BeautifulSoup(html,'html.parser')

In [ ]:
# prod_items=soup.select('div.main_prodlist > ul.product_list > li.prod_item > div.prod_main_info')
# len(prod_items)

In [ ]:
# title = prod_items[0].select('p.prod_name > a')[0].text.strip()
# print(title)

In [ ]:
# spec = prod_items[0].select('div.spec_list')[0].text.strip()
# print(spec)

In [ ]:
# price = prod_items[0].select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(",","")
# print(price)

In [ ]:
from tqdm import tqdm
import time

query="블루투스 헤드폰"
total_page=10
data_list=[]
for page in tqdm(range(1,total_page+1)):
  url = get_page(query,page)
  browser.get(url)
  time.sleep(3)
  
  html = browser.page_source
  soup=BeautifulSoup(html,'html.parser')
  prod_items=soup.select('div.main_prodlist > ul.product_list > li.prod_item > div.prod_main_info')
  for item in prod_items:
    title = item.select('p.prod_name > a')[0].text.strip()
    spec = item.select('div.spec_list')[0].text.strip()
    price = item.select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(",","")
    data_list.append([title,spec,price])
  print(len(data_list))
import pandas as pd
data_df = pd.DataFrame(data_list)
data_df.columns=['product_name','spec','price']
data_df.to_excel('./danawa_crawling_result.xlsx',index=False)

In [ ]:
import pandas as pd

data = pd.read_excel('./danawa_crawling_result.xlsx')
data.info()
data.head()

In [ ]:
company_list=[]
product_list=[]

for title in data['product_name']:
  splited = title.split(' ',1)
  company_name= splited[0]
  product_name = None
  if len(splited) > 1:
    product_name= splited[1]
  company_list.append(company_name)
  product_list.append(product_name)

In [ ]:
spec_list = data['spec'][0].split('/')
spec_list

In [ ]:
for spec in spec_list:
  if '재생시간' in spec:
    play_time = spec
  elif '무게' in spec:
    weight = spec
  elif '블루투스' in spec:
    bluetooth_version = spec
print(play_time, weight, bluetooth_version)

In [ ]:
play_time_number = play_time.split(' ')[1].strip()
weight_number = weight.split(' ')[1].strip()
bluetooth_version_number = bluetooth_version.split(' ')[1].strip()
print(play_time_number, weight_number, bluetooth_version_number)

In [ ]:
category_list=[]
play_time_list=[]
weight_list=[]
bluetooth_version_list=[]
for spec_str in data['spec']:
  spec_list = spec_str.split('/')
  category = spec_list[0]
  category_list.append(category)
  
  play_time_number = None
  weight_number = None
  bluetooth_version_number = None
  for spec in spec_list:
    if '재생시간' in spec:
      play_time_number = spec.split(' ')[1].strip()
    elif '무게' in spec:
      weight_number = spec.split(' ')[1].strip()
    elif '블루투스' in spec:
      if len(spec.split(' '))>1:
        bluetooth_version_number = spec.split(' ')[1].strip()
  play_time_list.append(play_time_number)    
  weight_list.append(weight_number)    
  bluetooth_version_list.append(bluetooth_version_number)  

In [ ]:
play_time_list[0:5]
weight_list[0:5]
bluetooth_version_list[0:5]

In [ ]:
import re
def preprocess_play_time(time_str):
  # print('time_str',time_str)
  result = None
  if time_str:
    reg = re.compile('(\d+)(?=시간)')
    result = reg.search(time_str)
 
    return result.group() if result else None
  else:
    return result

print(preprocess_play_time('20-40시간(abc)'))
print(preprocess_play_time('최대 11시간'))

In [ ]:
preprocess_play_time_list=[]
for time_str in play_time_list:
  # print('time_str',time_str)
    time = None
    result = preprocess_play_time(time_str)
    # print(result)
    if result:
      time = result
    preprocess_play_time_list.append(time)
  
preprocess_play_time_list[0:10]

In [ ]:
data_df = pd.DataFrame()
data_df['category'] = category_list
data_df['company'] = company_list
data_df['product'] = product_list
data_df['price'] = data['price']
data_df['play_time']=preprocess_play_time_list
data_df['weight']=weight_list
data_df['bluetooth_version']=bluetooth_version_list
data_df.head()

In [ ]:
data_df['category'].value_counts()

In [ ]:
data_df_final=data_df[data_df['category'].isin(['유선헤드폰']) == False]
len(data_df_final)

In [ ]:
data_df_final['price'].unique()

In [ ]:
data_df_final.to_excel('./danawa_data_preprocessing.xlsx', index=False)